In [13]:
import torch
import torchvision
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from torch.optim import SGD


In [5]:
device = 'cuda'
data_folder = '~/data/FMNIST'

In [6]:
fmnist = torchvision.datasets.FashionMNIST(data_folder, download=True, \
 train=True)

In [7]:
tr_images = fmnist.data
tr_targets = fmnist.targets

In [8]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()
        x = x.view(-1, 28 * 28)
        self.x , self.y = x, y
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x.to(device), y.to(device)
    def __len__(self):
        return len(self.x)
        

In [10]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    train_dataloader = DataLoader(train, batch_size=32, shuffle=True)
    return train_dataloader

In [14]:
def get_model():
    model = nn.Sequential(
        nn.Linear(in_features=28 * 28, out_features=1000),
        nn.ReLU(),
        nn.Linear(in_features=1000, out_features=10)
    ).to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=1e-2)
    return model, loss_function, optimizer

In [15]:
def train_batch(x, y, model, optimizer, loss_function):
    model.train()
    prediction = model(x)
    batch_loss = loss_function(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.items()


In [16]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()